In [1]:
import threading
from threading import Thread

import logging

FORMAT = '%(asctime)-15s %(threadName)s %(message)s'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
def where_am_i():
    print("Now I am running %s the main thread" % ('IN ' if threading.current_thread() == threading.main_thread() else 'NOT IN '))

def heavy_work():
    int_log = logging.getLogger(__name__)
    where_am_i()
    print("Performing some actions ... ")
    int_log.info("Exit the thead")
    
    
where_am_i()
    
t = Thread(name='MyThread', target=heavy_work)
t.start()




INFO:__main__:Exit the thead


Now I am running IN  the main thread
Now I am running NOT IN  the main thread
Performing some actions ... 


In [4]:

class MyThread(Thread):
    def __init__(self):
        super().__init__()
        
    def run(self):
        print("This is actually what this thread is doing...")
        
        
mt = MyThread()
mt.start()

This is actually what this thread is doing...


In [14]:

## A bus seats booking
## Actions: Book, pay, confirm


def remaining_seats(initial):
    if initial <= 0:
        return None
    
    remaining = initial
    while remaining > 0:
        yield "#{0:03d}".format(remaining)
        remaining -= 1
        
    return None

for seat in (x for x in remaining_seats(10)):
    print(seat)


#010
#009
#008
#007
#006
#005
#004
#003
#002
#001
